In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
data_dir = "./MNIST_data"
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# configurations
learning_rate = 0.001
batch_size =256
training_iters = 30000*batch_size
display_step = 100
testing_step = 1000
n_input = 28
n_steps = 28 #timestep
n_hidden = 128#hidden layers
n_classes =10

In [4]:
# shape
mnist.train.images.shape
mnist.train.labels.shape

(55000, 10)

In [5]:
x=tf.placeholder (tf.float32,[None,n_steps,n_input],name='X')
y=tf.placeholder (tf.float32,[None,n_classes],name='Y')
weights ={'out':tf.Variable(tf.random_normal([n_hidden,n_classes]),name='Weight')}
biases ={'out':tf.Variable(tf.random_normal([n_classes]),name='bias')}

In [6]:
# define RNN
def RNN(x,weights,bias):
    x=tf.unstack(x,n_steps,1)
    lstm_cell=rnn.BasicLSTMCell(n_hidden,forget_bias=0.8)
    lstm_cell=rnn.DropoutWrapper(lstm_cell)
    outputs, states =rnn.static_rnn(lstm_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out']+biases['out'])

In [7]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y),name=None)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
init = tf.global_variables_initializer()

In [8]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./Summary',sess.graph)
    step = 1
    while step*batch_size < training_iters:
        batch_x,batch_y =mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape(batch_size,n_steps,n_input)
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y})
        if step % display_step ==0:
            acc = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            loss = sess.run(cost,feed_dict={x:batch_x,y:batch_y})
            print('Iter:'+str(step)+', Minibatch Loss='+\
             '{:.6f}'.format(loss)+', Trainging Accurary='+\
             '{:.5f}'.format(acc))
        if step % testing_step == 0:
        
            # validation performance
            batch_x,batch_y =mnist.validation.next_batch(batch_size)
            batch_x = batch_x.reshape(batch_size,n_steps,n_input)
            acc__ = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            
            # test performance
            batch_x,batch_y =mnist.test.next_batch(batch_size)
            batch_x = batch_x.reshape(batch_size,n_steps,n_input)
            acc_ = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            
            print ("========> Validation Accuarcy: " + "{:.6f}".format(acc__) \
            + ", Testing Accuarcy: " + "{:.6f}".format(acc_)) 
            print("\n")
        
        step+=1
    print('Optimization Finished!')
    writer.close()
    test_len =128
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps,n_input))
    test_label = mnist.test.labels[:test_len]
    print('========> Final Test Accuracy:', \
          sess.run(accuracy,feed_dict={x:test_data,y:test_label}))

Iter:100, Minibatch Loss=0.427705, Trainging Accurary=0.84766
Iter:200, Minibatch Loss=0.269921, Trainging Accurary=0.93750
Iter:300, Minibatch Loss=0.199076, Trainging Accurary=0.94531
Iter:400, Minibatch Loss=0.087399, Trainging Accurary=0.97656
Iter:500, Minibatch Loss=0.085337, Trainging Accurary=0.97656
Iter:600, Minibatch Loss=0.089752, Trainging Accurary=0.97266
Iter:700, Minibatch Loss=0.056889, Trainging Accurary=0.96875
Iter:800, Minibatch Loss=0.088838, Trainging Accurary=0.96875
Iter:900, Minibatch Loss=0.079320, Trainging Accurary=0.97656
Iter:1000, Minibatch Loss=0.037308, Trainging Accurary=0.98047
========> Validation Accuarcy: 0.968750, Testing Accuarcy: 0.972656


Iter:1100, Minibatch Loss=0.065583, Trainging Accurary=0.98438
Iter:1200, Minibatch Loss=0.103206, Trainging Accurary=0.97656
Iter:1300, Minibatch Loss=0.022645, Trainging Accurary=0.98828
Iter:1400, Minibatch Loss=0.044684, Trainging Accurary=0.99219
Iter:1500, Minibatch Loss=0.038566, Trainging Accurary=0.

Iter:11900, Minibatch Loss=0.000507, Trainging Accurary=1.00000
Iter:12000, Minibatch Loss=0.000616, Trainging Accurary=1.00000
========> Validation Accuarcy: 0.992188, Testing Accuarcy: 0.976562


Iter:12100, Minibatch Loss=0.000868, Trainging Accurary=1.00000
Iter:12200, Minibatch Loss=0.000340, Trainging Accurary=1.00000
Iter:12300, Minibatch Loss=0.000386, Trainging Accurary=1.00000
Iter:12400, Minibatch Loss=0.000753, Trainging Accurary=1.00000
Iter:12500, Minibatch Loss=0.001961, Trainging Accurary=1.00000
Iter:12600, Minibatch Loss=0.000202, Trainging Accurary=1.00000
Iter:12700, Minibatch Loss=0.000080, Trainging Accurary=1.00000
Iter:12800, Minibatch Loss=0.002861, Trainging Accurary=1.00000
Iter:12900, Minibatch Loss=0.000421, Trainging Accurary=1.00000
Iter:13000, Minibatch Loss=0.000186, Trainging Accurary=1.00000
========> Validation Accuarcy: 0.988281, Testing Accuarcy: 0.984375


Iter:13100, Minibatch Loss=0.002913, Trainging Accurary=1.00000
Iter:13200, Minibatch Loss=0

Iter:23400, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:23500, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:23600, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:23700, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:23800, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:23900, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:24000, Minibatch Loss=0.000000, Trainging Accurary=1.00000
========> Validation Accuarcy: 0.988281, Testing Accuarcy: 0.988281


Iter:24100, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:24200, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:24300, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:24400, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:24500, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:24600, Minibatch Loss=0.000000, Trainging Accurary=1.00000
Iter:24700, Minibatch Loss=0.000001, Trainging Accurary=1.00000
Iter:24800, Minibatch Loss=0.00000